# TF-IDF

We'll rely on term frequency times inverted document frequency ([TF-IDF](https://web.stanford.edu/~jurafsky/slp3/11.pdf)), a powerful implementation of the [bag-of-words model](https://web.stanford.edu/~jurafsky/slp3/B.pdf), to measure meaningful similarity between documents while disregarding word order. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

In [ ]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def normalize(target):
    # This dict limits orthographical variation beyond the rule sets
    # of stjorn-extract.ipynb and menota-extract.ipynb:
    matrix = {
        'j': 'i',
        'v': 'u',
        # Experiment with *either* normalizing ð to þ,
        # or else both to d (because AM 226 often uses d for ð):
        'ð': 'þ',
        #'ð': 'd',
        #'þ': 'd',
        'á': 'a',
        'ǽ': 'æ',
        'ę': 'æ',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ý': 'y',
        'ǿ': 'ø',
        'k': 'c', # rather than vice versa, because of Latin (e.g. Lucifer)
        '[': '',
        ']': ''
        }
    for k,v in matrix.items():
        target = target.replace(k, v)
    return target

titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]]))
    else:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:]))

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        # We'll subject Menota to the same normalization standard as Stjórn:
        menota[ref] = normalize(doc.read().replace('\n', ''))

Note the arguments passed to the vectorizer class below. `min_df` sets a cutoff for the minimum number of documents in which a term has to appear in order to be included in the model. Since terms exclusive to single compositions are among the things that interest us, we'll leave this at `1`. `max_df` sets a cutoff point above which relative document frequency a term is ignored: in other words, a value of `0.8` ignores words occurring in over 80 percent of documents. Changing this value massively changes the document similarity scores downstream, and drastic changes in the setting have a pronounced effect on the document similarity rankings as well. A strict threshold is at any rate required to gain an insight into the relevance of individual terms, as leaving `max_df` at its default of `1.0` would lead the model to conclude that "oc" is the most meaningful term in many of our documents, while "æigi" is the top-ranking term for _Stjórn III_ as well as the _Norwegian Homily Book_ with a value as low as `0.4`. We can therefore leave the score high in the _Stjórn_-internal comparison, but may want to set a lower threshold for the larger corpus.

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.8)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

,stjorn1,stjorn2,stjorn3,stjorn4
stjorn1,1.000000,0.063975,0.510216,0.405235
stjorn2,0.063975,1.000000,0.054947,0.057555
stjorn3,0.510216,0.054947,1.000000,0.334973
stjorn4,0.405235,0.057555,0.334973,1.000000


After eliminating such variation as vowel length marks and the þ/ð distinction, these are now all pretty similar to one another, with the biggest difference between _Stjórn II_ and _III_.

Now let's first add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [4]:
# We want only those parts of Unger's NHB matched in Menota:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1']
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + normalize(doc.read().replace('\n', ''))
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

,stjorn1,stjorn2,stjorn3,stjorn4,ks,nhb
stjorn1,1.000000,0.308887,0.211326,0.370354,0.010920,0.045486
stjorn2,0.308887,1.000000,0.145243,0.358827,0.067233,0.022487
stjorn3,0.211326,0.145243,1.000000,0.209011,0.219577,0.414135
stjorn4,0.370354,0.358827,0.209011,1.000000,0.000887,0.056356
ks,0.010920,0.067233,0.219577,0.000887,1.000000,0.229910
nhb,0.045486,0.022487,0.414135,0.056356,0.229910,1.000000


_Stjórn III_ and _Konungs skuggsjá_ share material cognate within the vernacular, but not so much, or with insufficient spelling agreement, to stand out in this matrix. In fact, the _Norwegian Homily Book_ has a higher match with _Stjórn III_ than _Konungs skuggsjá_ does, which may be explained at least in part by the closer subject match for those parts of _Stjórn III_ not reflected in _Konungs skuggsjá_.

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition as a proof of method for now.

In [5]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.7)
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df.sort_values(by=['stjorn3'], ascending=False)

,stjorn1,stjorn2,stjorn3,stjorn4,nhb,nraNorrFragm75_kross_saga,am132_egils_saga,am162btheta_njals_saga,nraNorrFragm64_barlaams_saga,nraNorrFragm81A_benedikts_regla,am1056IX_konungs_skuggsja_fragment,am78_kristinrettir,am63_heimskringla3,dg4-7_strengleikar,am132_droplaugasona_saga,am132_kormaks_saga,nraNorrFragm72x76_dialogar,nraNorrFragm53_haralds_saga_hardrada,am132_finnboga_saga,nraNorrFragm70_agotu_saga,dg4-7_pamphilius_saga,nraNorrFragm62_karlamagnuss_saga,dg4-7_eliss_saga,nraNorrFragm60A_stjorn,am132_fostbraedra_saga,lbsFragm82_olafs_saga_helga,nraNorrFragm58B_konungs_skuggsja,nraNorrFragm60C_stjorn,holmPerg30_landslog,am619_norwegian_homily_book,nraNorrFragm57_jons_saga_helga,nraNorrFragm69_nikulass_saga,am56_landslog,wolfAug9-10_egils_saga,nraNorrFragm66_thomass_saga,holmPerg17_thomass_saga,am383I_thorlaks_saga,holmPerg4_thidreks_saga,am132_njals_saga,am36_heimskringla2,am544_voluspa,am162bkappa_njals_saga,am305_landslog,nraNorrFragm58C_konungs_skuggsja,am132_olkofra_thattr,nraNorrFragm54_sverris_saga,nraNorrFragm55B_hakonar_saga,nraNorrFragm79_mariu_saga,gks2365_voluspa,am243balpha_konungs_skuggsja,nraNorrFragm51_fagrskinna,am132_viga-glums_saga,am279a_gragas,am677_gregory,am132_laxdoela_saga,am302_landslog,am178_thidreks_saga,am113b_islendingabok,nraNorrFragm81B_benedikts_regla,am132_bandamanna_saga,nraNorrFragm71_gregors_saga_pafa,am655_laeknisbok,am519a_alexanders_saga,holmPerg34_landslog,am162balpha_njals_saga,nraNorrFragm7_landslog,nraNorrFragm67_thomass_saga,nraNorrFragm56_thorgils_saga,nks235g_konungs_skuggsja,am132_hallfredar_saga,am35_heimskringla1,am242_codex_wormianus,meII2_frostathingslog,nraNorrFragm78_mariu_saga,dg8II_olafs_saga,nraNorrFragm80_pals_saga,nraNorrFragm63_karlamagnuss_saga,nraNorrFragm77_dialogar,am28_codex_runicus,holmPerg34_boejarlog,dg8I_landslog,nraNorrFragm60B_stjorn,nraNorrFragm55A_hakonar_saga,skbA120_marys_complaint,nraNorrFragm59_rimbegla,nraNorrFragm65_floress_saga,nraNorrFragm52_olafs_saga_helga_hin_elzta,holmPerg6_barlaams_saga,nraNorrFragm68_brendanuss_saga,nraNorrFragm61_karlamagnuss_saga,nraNorrFragm58A_konungs_skuggsja
stjorn3,0.311182,0.219514,1.000000,0.285717,0.308298,0.051853,0.315115,0.150851,0.131202,0.044639,0.019026,0.109576,0.179500,0.284217,0.187201,0.121040,0.103604,0.164539,0.150069,0.051606,0.225499,0.223550,0.253942,0.058605,0.199703,0.202287,0.184094,0.234922,0.096487,0.302921,0.149350,0.135471,0.110962,0.329670,0.169093,0.207535,0.051131,0.308221,0.250663,0.200322,0.088958,0.085066,0.049603,0.054939,0.149192,0.208857,0.108692,0.155278,0.058133,0.174281,0.043339,0.221837,0.061055,0.083400,0.264872,0.051185,0.038565,0.047312,0.108638,0.158565,0.087231,0.049090,0.189452,0.078972,0.070448,0.144056,0.250753,0.148263,0.082328,0.192172,0.188593,0.301264,0.122493,0.187897,0.178733,0.181360,0.187684,0.191177,0.028564,0.054698,0.134532,0.152632,0.227130,0.022186,0.046605,0.074384,0.140658,0.186920,0.095261,0.147208,0.096194
wolfAug9-10_egils_saga,0.159822,0.130853,0.329670,0.145503,0.150120,0.037293,0.726285,0.134570,0.066862,0.029944,0.008510,0.063357,0.182126,0.187030,0.213219,0.125850,0.087182,0.199553,0.143565,0.021417,0.119238,0.163334,0.138182,0.019941,0.183264,0.266295,0.069000,0.131074,0.061714,0.151433,0.149808,0.138662,0.064008,1.000000,0.107268,0.090672,0.048763,0.247308,0.242960,0.203925,0.048811,0.092972,0.027960,0.053269,0.146890,0.219993,0.106511,0.068655,0.044444,0.080408,0.016369,0.211566,0.063629,0.049593,0.288904,0.028131,0.033262,0.059261,0.051585,0.192278,0.064463,0.039393,0.136532,0.093356,0.082122,0.068482,0.151610,0.295589,0.039866,0.191488,0.211447,0.243896,0.074440,0.134015,0.116395,0.118662,0.156265,0.145169,0.009016,0.067209,0.070302,0.056526,0.270869,0.008079,0.042484,0.073365,0.166711,0.093493,0.068212,0.130578,0.059732
am132_egils_saga,0.151675,0.125608,0.315115,0.126290,0.095964,0.036256,1.000000,0.161351,0.047554,0.023609,0.009822,0.033274,0.202871,0.192585,0.289021,0.190429,0.108581,0.236492,0.221019,0.022717,0.074240,0.140000,0.115386,0

The score for the two editions of the _Norwegian Homily Book_ may serve as our proof of method: with any `max_df` setting, and whether or not we normalize &lt;þ&gt; and &lt;ð&gt; to &lt;d&gt;, these come to a similarity in the range 0.93~0.97. As this compares an edition of Unger's with a Menota transcription, as does our comparison of _Stjórn_ with the remainder of the Menota corpus, we may be confident that the scores give a fair indication of lexical similarity to the extent the TF-IDF measure can provide one.

Let's take the constituent parts of _Stjórn_ one at a time and rank the Menota material by similarity. At this point, having demonstrated the validity of our method at least for editions of the same manuscript, we may remove Unger's _Homily Book_ from the data set:

In [6]:
nhb_index = titles.index('nhb')
corpus.pop(nhb_index)
titles.pop(nhb_index)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

def rank(subject):
    sorted_df = df.sort_values([subject], ascending=False)
    print(sorted_df[subject][:13].to_string()) # Limiting output for brevity

In [7]:
for i in work_indices.keys():
    print(f"{i}:")
    print("--------------------------------------------")
    rank(i)
    print('')

stjorn1:
--------------------------------------------
stjorn1                             1.000000
stjorn4                             0.398167
stjorn2                             0.329877
stjorn3                             0.311565
am242_codex_wormianus               0.208877
nraNorrFragm58B_konungs_skuggsja    0.199956
holmPerg17_thomass_saga             0.175020
dg4-7_strengleikar                  0.165271
nraNorrFragm54_sverris_saga         0.164292
wolfAug9-10_egils_saga              0.160967
nraNorrFragm63_karlamagnuss_saga    0.160011
nraNorrFragm80_pals_saga            0.157136
am132_egils_saga                    0.152953

stjorn2:
--------------------------------------------
stjorn2                             1.000000
stjorn4                             0.359272
stjorn1                             0.329877
stjorn3                             0.219284
nraNorrFragm58B_konungs_skuggsja    0.187332
holmPerg34_landslog                 0.175640
am132_njals_saga                    

These are complex results. The redaction of _Thómass saga erkibyskups_ transmitted in Holm. Perg. 17 was "uden Tvivl tilbleven i Norge" as judged by its language and syntax, but probably later in the second half of the 13th century (Unger iii); "Oversættelsens Stil minder undertiden om Kongespeilet" (ibid.). The fact of its Norwegian origin alone does not explain an affinity with _Stjórn I_ against _Stjórn III_ (both of whose Norwegian ancestry is thought to be at two generations' remove at least), while the date of _Thomass saga_ would rather associate it with the latter. As for the similarity of style between _Thómass saga_ and _Konungs skuggsjá_ as observed by Unger, he cannot have meant a matter of style reflected in the cosine similarity of their TF-IDF within this corpus, as that is decidedly low:

In [8]:

df['holmPerg17_thomass_saga'].loc['nks235g_konungs_skuggsja']

0.058561162086225624

Since TF-IDF is a reflection of unusual word forms, our next step is to investigate which forms are conspicuously associated with the constituent parts of _Stjórn_. The outcome of this query is even more greatly affected by our `max_df` setting, and this is the test that prompted the equation between the characters &lt;þðd&gt; in preprocessing above, as Unger edited _Stjórn I_ from AM 226, which tends to use &lt;d&gt; for &lt;ð&gt;. This, as well as other normalization strategies in this notebook and others it depends on, should be kept in mind when consulting the below rankings of the most striking forms in each constituent part of _Stjórn_. It should also be remembered that commonly spelled function words have been eliminated using the `max_df` setting:

In [9]:
scores = pd.DataFrame(model.toarray(), titles, vectorizer.get_feature_names_out())
for i in work_indices.keys():
    print(f"{i}:")
    print("------------------------")
    print(scores.loc[i].sort_values(ascending=False)[:20])
    print('')

stjorn1:
------------------------
medr      0.531886
aa        0.307348
edr       0.232817
fyrir     0.225208
þi        0.187701
gud       0.178210
þiat      0.171045
eptir     0.127508
hafdi     0.107135
honum     0.099942
meþr      0.099569
man       0.091378
scylld    0.090109
þers      0.088724
guds      0.088033
þeirra    0.087901
tima      0.083897
taladi    0.083809
sagdi     0.082886
uaru      0.082568
Name: stjorn1, dtype: float64

stjorn2:
------------------------
aa          0.384853
þier        0.269518
gud         0.252151
moyses      0.240537
med         0.215359
firir       0.199686
uit         0.197960
suo         0.183912
mun         0.182671
mællti      0.168344
scaltu      0.154965
aaron       0.153237
yfir        0.139124
drottinn    0.136138
eda         0.107792
cyni        0.107689
brutt       0.101818
balaam      0.095157
moysen      0.094956
ydr         0.094455
Name: stjorn2, dtype: float64

stjorn3:
------------------------
æigi        0.264773
dauid       0.2

_Stjórn I_ stood out for its form "medr" *before* the conflation of &lt;þðd&gt;, but it continues to be its most remarkable form after, occurring as it does in 16 out of 85 Menota items. What is also striking is that the exact form "medr" so frequently found in AM 226 occurs three times in the Holm. Perg. 17 manuscript of _Thómass saga erkibyskups_, while the AM 132 text of _Laxdœla saga_ and the Norr. Fragm. 58B fragment of _Konungs skuggsjá_ have it once each. "Medr" is thus not only a form that sets (the AM 226 text of) _Stjórn I_ apart, it is also one of the forms associating it with _Thómass saga_.